# Predykcja wytrzymałości na ściskanie na podstawie rzeczywistych danych wpisanych

Za pomocą bibloteki "pickle" model został wyciągnięty oraz zserializowany w celu dalszego użytkowania. Dużym plusem jest to że nie potrzeba instalować dodatkowych bibliotek oprócz standardowej biblioteki "pickle"


In [168]:
import pickle
import pandas as pd

W tym kroku przypiszemy wartości poszczególnych składników mieszaki betonowej

In [169]:
rows = [430,0, 0, 178.5, 8.0, 1001.9, 688.7, 28]

In [170]:
columns = ['cement [kg/m3]',"żużel_wielkopiecowy [kg/m3]", 'popiół_lotny [kg/m3]', 'woda [kg/m3]',
       'superplastyfikator [kg/m3]', 'kruszywo_grube [kg/m3]',
       'kruszywo_drobne [kg/m3]', 'wiek [dni]']

In [171]:
predicted = pd.DataFrame([dict(zip(columns,rows))])
predicted

,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
0,430,0,0,178.5,8.0,1001.9,688.7,28


Otwieramy teraz zapisany wcześniej wytrenowany model

In [172]:
with open (r"C:\Users\Damian\Desktop\xD\Projekty\concrete-strenght-calculator\modelPickle", "rb") as model_dat:
    model = pickle.load(model_dat)

I podstawiamy nasze przykładowe dane do wytrenowanego modelu

In [173]:
print(model.predict(predicted)[0], "MPa")

52.3 MPa


Teraz zmniejszymy zawartość cementu przy jednoczesnym dodaniu iliości popiołu 

In [174]:
predicted.loc[0,"cement [kg/m3]"] = 350
predicted.loc[0,"popiół_lotny [kg/m3]"] = 140

In [175]:
predicted

,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
0,350,0,140,178.5,8.0,1001.9,688.7,28


I ponownie przeliczamy nasz model

In [176]:
print(model.predict(predicted)[0], "MPa")

37.27 MPa
